# Polynomial Resampling

## Exploring algorithms for real-time resampling/interpolation of sensor-data.

As opposed to batchwise resampling, where you have the entire time-series with many datapoints available to recreate the signal, we are assuming that we only have a short sliding window of datapoint. This is a result of having the resampling algorithm be highly performant, and also of having to resample in real-time on an incoming stream of data.

We are also assuming that the incoming data is non-uniform in the sense that the data is not coming at specified regular interval. Some of the data might also be missing at the time when resampling has to be performed.

This document will explore some of the possibilities and options for resampling by fitting the datapoints with piecewise polynomial interpolation functions.

### Choosing the Polynomial Degree

A linear fit between datapoints is the simplest solution to implement. And one that requires the least amount of data to perform. All that is needed are two points to interpolate between. As the degree of the polynomial increases more data is required, as the polynomial has free parameters that must be determined. But, higher polynomial degree also yields better fitting curves with smoother transitions between segments of the piecewise interpolation.

We argue that cubic polynomials are the best choice as this is the smallest polynomial degree which allows one to not only align the points, but also align the first derivates resulting in smooth transisition from one polynomial piece to another.

In [78]:
import numpy as np

from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models.widgets import Slider
from bokeh.layouts import row, widgetbox
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()


def modify_doc(doc):
   

    # Create the main plot
    def create_figure():
        # Set up data
        t_in = np.array([0., 1.])
        f_in = np.array([2., 3.])
        diff_in = np.array([0., 0.])

        # cubic polynomialS
        polyDeg = 3

        A = np.zeros([polyDeg + 1, polyDeg + 1])
        for i in np.arange(0, polyDeg + 1):
            A[[0, 1], i] = np.power(t_in, polyDeg - i)

        A[2, :] = np.array([3 * np.square(t_in[0]), 2 * t_in[0], 1, 0])
        A[3, :] = np.array([3 * np.square(t_in[1]), 2 * t_in[1], 1, 0])

        t = np.arange(0, 1, 0.01)

        # Get data
        f_in[0] = f1_slider.value
        f_in[1] = f2_slider.value

        diff_in[0] = diff1_slider.value
        diff_in[1] = diff2_slider.value

        # Calculate polynomial
        cubicParams = np.dot(np.linalg.inv(A), np.append(f_in, diff_in))
        y = np.polyval(cubicParams, t)
        
        # Caluclate linear diffs
        t1 = t_in[0];
        f1 = f_in[0];
        d1 = diff_in[0];
        l1 = 0.2*np.cos(np.arctan(d1))
        
        t1vec = np.arange(t1-l1,t1+l1, 0.01)
        y1vec = f1 + d1*(t1vec-t1)
        
        t2 = t_in[1];
        f2 = f_in[1];
        d2 = diff_in[1];
        l2 = 0.2*np.cos(np.arctan(d2))
        
        t2vec = np.arange(t2-l2,t2+l2, 0.01)
        y2vec = f2 + d2*(t2vec-t2)

        # Set up Plot
        plot = figure(plot_width=400, plot_height=400, x_range=[-0.3,1.3], y_range=[0.7,4.3])
        plot.line(t, y, line_width=1, color="blue")
        plot.circle(t_in, f_in, size=8, fill_color="white")
        
        plot.line(t1vec, y1vec, color="red", line_dash="dotted")
        plot.line(t2vec, y2vec, color="red", line_dash="dotted")
            
        return plot


    # Update the plot
    def update(att, old, new):
        f_in = [f1_slider.value, f2_slider.value]
        layout.children[1] = create_figure()

    # Controls
    f1_slider = Slider(start=1, end=4, value=2, step=0.01, title="f(0)")
    f2_slider = Slider(start=1, end=4, value=3, step=0.01, title="f(1)")
    diff1_slider = Slider(start=-10, end=10, value=0, step=0.01, title="f'(1)")
    diff2_slider = Slider(start=-10, end=10, value=0, step=0.01, title="f'(2)")
    
    for widget in [f1_slider, f2_slider, diff1_slider, diff2_slider]:
        widget.on_change('value', update)
    
    inputs = widgetbox(f1_slider, f2_slider, diff1_slider, diff2_slider)
    layout = row(inputs, create_figure())

    doc.add_root(layout)
    doc.title = "Polynomial Fit"
    
handler = FunctionHandler(modify_doc)
app = Application(handler)

doc = app.create_document()
show(app, notebook_url="localhost:8888")

Loading BokehJS ...

In [80]:


def modify_doc3(doc):
    
    # Set up data
    t_in = np.array([0., 60., 120., 180.])
    f_in = np.array([0., 1., -1., 0])
    stage_in = 1
    diff = np.array([0., 0., 0., 0.,])
    
        
    polyDeg = 3;
    arraySize = polyDeg+1
    step_size = 1
    interpolation_step = 20
    
    A1 = np.zeros([arraySize, arraySize])
    for i in np.arange(0, arraySize):
        A1[[0, 1], i] = np.power(t_in[[0,1]], polyDeg - i)

    A1[2, :] = np.array([3 * np.square(t_in[0]), 2 * t_in[0], 1, 0])
    A1[3, :] = np.array([3 * np.square(t_in[1]), 2 * t_in[1], 1, 0])
    
    A2 = np.zeros([arraySize, arraySize])
    for i in np.arange(0, arraySize):
        A2[[0, 1], i] = np.power(t_in[[1,2]], polyDeg - i)

    A2[2, :] = np.array([3 * np.square(t_in[1]), 2 * t_in[1], 1, 0])
    A2[3, :] = np.array([3 * np.square(t_in[2]), 2 * t_in[2], 1, 0])
    
    A3 = np.zeros([arraySize, arraySize])
    for i in np.arange(0, arraySize):
        A3[[0, 1], i] = np.power(t_in[[2,3]], polyDeg - i)

    A3[2, :] = np.array([3 * np.square(t_in[2]), 2 * t_in[2], 1, 0])
    A3[3, :] = np.array([3 * np.square(t_in[3]), 2 * t_in[3], 1, 0])
    
    def calculate_polynomial(A, f, df, t):
        cubic_params = np.dot(np.linalg.inv(A), np.append(f, df))
        y = np.polyval(cubic_params, t)
        return y
        
    def create_figure():
        
        # Plotting parameters
        vec_length = 10
        
        # Set up plot
        plot = figure(plot_width=600, height=300, x_range=[-10,190], y_range=[-1.1,1.1])
        
        # Calculate weight-averaged differentials
        
        w0 = t_in[1]-t_in[0]
        w1 = t_in[2]-t_in[1]
        w2 = t_in[3]-t_in[2]
                
        if stage_in == 1:
            
            diff[0] = (f_in[1]-f_in[0])/w0
            diff[1] = diff[0]
            
            # Calculate polynomials
            t1 = np.arange(t_in[0], t_in[1], step_size)
            y1 = calculate_polynomial(A1, f_in[[0,1]], diff[[0,1]], t1)
                        
            t1interp = np.extract(np.mod(t1, interpolation_step)==0, t1)
            y1interp = calculate_polynomial(A1, f_in[[0,1]], diff[[0,1]], t1interp)
            
            alpha = np.arctan(diff[0])
            tdiff1 = [t_in[0] - vec_length * np.cos(alpha), t_in[0]+vec_length*np.cos(alpha)]
            ydiff1 = [f_in[0] - vec_length * np.sin(alpha), f_in[0]+vec_length*np.sin(alpha)]
            
            alpha = np.arctan(diff[1])
            tdiff2 = [t_in[1] - vec_length * np.cos(alpha), t_in[1]+vec_length*np.cos(alpha)]
            ydiff2 = [f_in[1] - vec_length * np.sin(alpha), f_in[1]+vec_length*np.sin(alpha)]
            
            plot.line(t1, y1, line_width=3, line_alpha=0.6)
            plot.circle(t1interp, y1interp, size=4)
            plot.line(tdiff1,ydiff1, color="red", line_width=2, line_dash="dotted")
            plot.line(tdiff2,ydiff2, color="red", line_width=2, line_dash="dotted")
            plot.circle(t_in[[0,1]], f_in[[0,1]],size=8, fill_color="white")
            
        elif stage_in == 2:
            
            # Calculate differentials            
            diff[0] = (f_in[1]-f_in[0])/w0
            diff[2] = (f_in[2]-f_in[1])/w1
            diff[1] = (diff[0]*w0+diff[2]*w1)/(w0+w1) # weighted average
            
            # Calculate first polynomial
            t1 = np.arange(t_in[0], t_in[1], step_size)
            y1 = calculate_polynomial(A1, f_in[[0,1]], diff[[0,1]], t1)
                        
            t1interp = np.extract(np.mod(t1, interpolation_step)==0, t1)
            y1interp = calculate_polynomial(A1, f_in[[0,1]], diff[[0,1]], t1interp)
            
            alpha = np.arctan(diff[0])
            tdiff1 = [t_in[0] - vec_length * np.cos(alpha), t_in[0]+vec_length*np.cos(alpha)]
            ydiff1 = [f_in[0] - vec_length * np.sin(alpha), f_in[0]+vec_length*np.sin(alpha)]
            
            alpha = np.arctan(diff[1])
            tdiff2 = [t_in[1] - vec_length * np.cos(alpha), t_in[1]+vec_length*np.cos(alpha)]
            ydiff2 = [f_in[1] - vec_length * np.sin(alpha), f_in[1]+vec_length*np.sin(alpha)]
            
            # Calculate second polynomial
            t2 = np.arange(t_in[1], t_in[2], step_size)
            y2 = calculate_polynomial(A2, f_in[[1,2]], diff[[1,2]], t2)
                        
            t2interp = np.extract(np.mod(t2, interpolation_step)==0, t2)
            y2interp = calculate_polynomial(A2, f_in[[1,2]], diff[[1,2]], t2interp)
            
            alpha = np.arctan(diff[2])
            tdiff3 = [t_in[2] - vec_length * np.cos(alpha), t_in[2]+vec_length*np.cos(alpha)]
            ydiff3 = [f_in[2] - vec_length * np.sin(alpha), f_in[2]+vec_length*np.sin(alpha)]
            
            plot.line(t1, y1, line_width=3, line_alpha=0.6)
            plot.circle(t1interp, y1interp, size=4)
            
            plot.line(t2, y2, line_width=3, line_alpha=0.6)
            plot.circle(t2interp, y2interp, size=4)
            
            plot.line(tdiff1,ydiff1, color="red", line_width=2, line_dash="dotted")
            plot.line(tdiff2,ydiff2, color="red", line_width=2, line_dash="dotted")
            plot.line(tdiff3,ydiff3, color="red", line_width=2, line_dash="dotted")
            plot.circle(t_in[[0,1,2]], f_in[[0,1,2]],size=8, fill_color="white")
            
        elif stage_in == 3:
            
            # Calculate differentials            
            diff[0] = (f_in[1]-f_in[0])/w0
            diff[3] = (f_in[3]-f_in[2])/w2
            
            tempdiff = (f_in[2]-f_in[1])/w1
            diff[1] = (diff[0]*w0+tempdiff*w1)/(w0+w1)
            diff[2] = (tempdiff*w1+diff[3]*w2)/(w1+w2) # weighted average
            
            # Calculate first polynomial
            t1 = np.arange(t_in[0], t_in[1], step_size)
            y1 = calculate_polynomial(A1, f_in[[0,1]], diff[[0,1]], t1)
                        
            t1interp = np.extract(np.mod(t1, interpolation_step)==0, t1)
            y1interp = calculate_polynomial(A1, f_in[[0,1]], diff[[0,1]], t1interp)
            
            alpha = np.arctan(diff[0])
            tdiff1 = [t_in[0] - vec_length * np.cos(alpha), t_in[0]+vec_length*np.cos(alpha)]
            ydiff1 = [f_in[0] - vec_length * np.sin(alpha), f_in[0]+vec_length*np.sin(alpha)]
            
            alpha = np.arctan(diff[1])
            tdiff2 = [t_in[1] - vec_length * np.cos(alpha), t_in[1]+vec_length*np.cos(alpha)]
            ydiff2 = [f_in[1] - vec_length * np.sin(alpha), f_in[1]+vec_length*np.sin(alpha)]
            
            # Calculate second polynomial
            t2 = np.arange(t_in[1], t_in[2], step_size)
            y2 = calculate_polynomial(A2, f_in[[1,2]], diff[[1,2]], t2)
                        
            t2interp = np.extract(np.mod(t2, interpolation_step)==0, t2)
            y2interp = calculate_polynomial(A2, f_in[[1,2]], diff[[1,2]], t2interp)
            
            alpha = np.arctan(diff[2])
            tdiff3 = [t_in[2] - vec_length * np.cos(alpha), t_in[2]+vec_length*np.cos(alpha)]
            ydiff3 = [f_in[2] - vec_length * np.sin(alpha), f_in[2]+vec_length*np.sin(alpha)]
            
            # Calculate third polynomial
            t3 = np.arange(t_in[2], t_in[3], step_size)
            y3 = calculate_polynomial(A3, f_in[[2,3]], diff[[2,3]], t3)
                        
            t3interp = np.extract(np.mod(t3, interpolation_step)==0, t3)
            y3interp = calculate_polynomial(A3, f_in[[2,3]], diff[[2,3]], t3interp)
            
            alpha = np.arctan(diff[3])
            tdiff4 = [t_in[3] - vec_length * np.cos(alpha), t_in[3]+vec_length*np.cos(alpha)]
            ydiff4 = [f_in[3] - vec_length * np.sin(alpha), f_in[3]+vec_length*np.sin(alpha)]
            
            # Plot the curves
            
            plot.line(t1, y1, line_width=3, line_alpha=0.6)
            plot.circle(t1interp, y1interp, size=4)
            
            plot.line(t2, y2, line_width=3, line_alpha=0.6)
            plot.circle(t2interp, y2interp, size=4)
            
            plot.line(t3, y3, line_width=3, line_alpha=0.6)
            plot.circle(t3interp, y3interp, size=4)
            
            plot.line(tdiff1,ydiff1, color="red", line_width=2, line_dash="dotted")
            plot.line(tdiff2,ydiff2, color="red", line_width=2, line_dash="dotted")
            plot.line(tdiff3,ydiff3, color="red", line_width=2, line_dash="dotted")
            plot.line(tdiff4,ydiff4, color="red", line_width=2, line_dash="dotted")
            plot.circle(t_in[[0,1,2,3]], f_in[[0,1,2,3]],size=8, fill_color="white")
                
       
        return plot
    
    # Update the plot
    def update(attr, old, new):
        f_in[0] = f1_slider.value
        f_in[1] = f2_slider.value
        f_in[2] = f3_slider.value
        f_in[3] = f4_slider.value
        

        nonlocal interpolation_step
        nonlocal stage_in
        
        interpolation_step = interpolation_slider.value
        stage_in = stage_slider.value
        
        layout.children[0] = create_figure()
    
    # Controls
    f1_slider = Slider(start=-1, end=1, value=0, step=0.1, title="f(t\u2080)")
    f2_slider = Slider(start=-1, end=1, value=1, step=0.1, title="f(t\u2081)")
    f3_slider = Slider(start=-1, end=1, value=-1, step=0.1, title="f(t\u2082)")
    f4_slider = Slider(start=-1, end=1, value=0, step=0.1, title="f(t\u2083)")
    
    interpolation_slider = Slider(start=2, end=40, value=20, step=2, title="interpolation step")
    stage_slider = Slider(start=1, end=3, value=1, step=1, title="stage")
    
    for widget in [f1_slider, f2_slider, f3_slider, f4_slider, interpolation_slider, stage_slider]:
        widget.on_change('value', update)
    
    inputs = widgetbox([f1_slider, f2_slider, f3_slider, f4_slider, interpolation_slider, stage_slider])
    layout = row(create_figure(), inputs)
    
    doc.add_root(layout)
    
handler3 = FunctionHandler(modify_doc3)
app = Application(handler3)

doc = app.create_document()
show(app, notebook_url="localhost:8888")